Install packages and mount google drive

In [ ]:
import os
!pip install torch torchvision torchaudio

import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

#!pip install torch-sparse #-f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install torch-scatter #-f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.3.0+cu121.html
#!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.3.0+cu121.html

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
#!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

!pip install torch-geometric
!pip install codetiming
!pip install wandb
!pip install plotly
!pip install line-profiler

In [ ]:
# provjera jel se sve dobro instaliralo
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)
tensor = torch.rand(3, 3).to(device)
print(tensor)

Initialize dataset

In [ ]:
!pip3 install codetiming
%run -i '/home/jovyan/MIDS-GNN/pyg_dataset_second.py'
#%run -i '/home/jovyan/MIDS_collection/PyTorch Geometric/Dataset/raw_file_snapshot.py'

Load dataset

In [ ]:
from pathlib import Path
import os
os.chdir('/home/jovyan/MIDS-GNN')
root ="/home/jovyan/MIDS-GNN/Dataset"

selected_graph_sizes =     {
                            #3: -1,
                            #4: -1,
                            #5: -1,
                            #6: -1,
                            #7: -1,
                            #8:  10000,
                            #9:  10000,
                            #10: 10000,
                            #11: 10000,
                            #12: 10000,
                            #13: 10000,
                            #14: 10000,
                            #15: 10000,
                            #20: 10000,
                            50: 2000,
                        }
loader = GraphDataset(selection=selected_graph_sizes)

#import zipfile
#with zipfile.ZipFile('/home/jovyan/MIDS_collection/PyTorch Geometric/Dataset/raw_data.zip', 'r') as zip_ref:
#    zip_ref.extractall('/home/jovyan/MIDS_collection/PyTorch Geometric/Dataset/raw_data')
    
raw_included_subdirs = None

dataset = MIDSdataset(root, loader)

print()
print(f"Dataset: {dataset}:")
print("====================")
print(f"Number of graphs: {len(dataset)}")
print(f"Number of features: {dataset.num_features}")
#print(f"Number of classes: {dataset.num_classes}")

Define GNN structure

In [ ]:
import os.path as osp
import time

import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score

from torch_geometric.datasets import PPI
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv

torch.manual_seed(12345)
dataset = dataset.shuffle()

# split dataset into train and test
train_dataset = dataset[:int(len(dataset)*0.8)] # train 80
print(len(train_dataset))
test_dataset = dataset[int(len(dataset)*0.8):] # test 20
print(len(test_dataset))

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(train_dataset.num_features, 256, heads=4)
        self.lin1 = torch.nn.Linear(train_dataset.num_features, 4 * 256)
        self.conv2 = GATConv(4 * 256, 256, heads=4)
        self.lin2 = torch.nn.Linear(4 * 256, 4 * 256)
        self.conv3 = GATConv(4 * 256, 256, heads=4)
        self.lin3 = torch.nn.Linear(4 * 256, 4 * 256)
        self.conv4 = GATConv(4 * 256, 1, heads=6,
                             concat=False)
        self.lin4 = torch.nn.Linear(4 * 256, 1)

    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x, edge_index) + self.lin1(x))
        x = F.elu(self.conv2(x, edge_index) + self.lin2(x))
        x = F.elu(self.conv3(x, edge_index) + self.lin3(x))
        x = self.conv4(x, edge_index) + self.lin4(x)
        return x.squeeze()

Train a model

In [ ]:
import os
import torch

from math import inf
import torch.nn.functional as F
from torch import Tensor
from typing import Optional

class CustomLossFunction(torch.nn.BCEWithLogitsLoss):
  def forward(self, input: Tensor, target: Tensor) -> Tensor:
      tens = torch.split(target, input.size(dim=0))[0]
      loss = F.binary_cross_entropy_with_logits(input, tens, self.weight, pos_weight=self.pos_weight, reduction=self.reduction)
      
      for tens in torch.split(target, input.size(dim=0)):
          new = F.binary_cross_entropy_with_logits(input, tens, self.weight, pos_weight=self.pos_weight, reduction=self.reduction)
          if new.item() < loss.item():
              loss = new
      return loss

from torch_sparse import SparseTensor

def check_MIDS(A, candidate, target_value):
    """
    Args:
        - A: adjacency matrix
        - candidate: node labels that are candidate for MIDS (data.y)
        - target_value: known size of the MIDS
    """
    # TODO: This function needs to be adjusted.
    #   - Instead of adjacency matrix, we may pass the edgelist and convert it here

    n = len(candidate)

    # Candidate set is not minimal
    if sum(candidate) > target_value:
        return False

    # Candidate set is not dominating and independent
    if not all((A + np.eye(n)) @ candidate >= 1):
        return False

    for i in range(n):
        for j in range(i+1, n):
            if candidate[i] and candidate[j] and A[i,j]:
                return False



    if sum(candidate) < target_value:
        print(f"Somehow we found an even smaller MIDS: {sum(candidate)}, {target_value}")
        pass

    return True

import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.cuda.device_count())
model = Net().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.00004)
#loss_op = torch.nn.CrossEntropyLoss()
#loss_op = torch.nn.BCEWithLogitsLoss()
loss_op = CustomLossFunction()


def train():
    model.train()

    total_loss = 0
    for i, data in enumerate(train_loader):
        data = data.to(device)

        optimizer.zero_grad()
        pred = model(data.x.type(torch.FloatTensor).to(device), data.edge_index)
        real = data.y.type(torch.LongTensor)
        loss = loss_op(pred.to(device), data.y.to(device))

        total_loss += loss.item() * data.num_graphs
        loss.backward()
        optimizer.step()

    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader):
    model.eval()
    flag = 0
    correct = 0
    correct_checkMIDS = 0
    for i, data in enumerate(loader):
        for idx in range(data.num_graphs):
            out = model(data.get_example(idx).x.type(torch.FloatTensor).to(device), data.get_example(idx).edge_index.to(device))
            #print(out)
          
            pred = torch.where(out > 0, 1.0, 0.0)
            #print(data.edge_index)
            pred_2 = torch.round(out)
            #correct += int(torch.equal(pred.to(device),data.get_example(idx).y.type(torch.LongTensor).to(device)))  # Derive ratio of correct predictions.

            num_nodes = data.get_example(idx).num_nodes
            adjacency = SparseTensor(row=data.get_example(idx).edge_index[0], col=data.get_example(idx).edge_index[1], sparse_sizes=(num_nodes, num_nodes))
            adjacency_matrix = adjacency.to_dense()
            #print(torch.split(data.get_example(idx).y, data.num_nodes))
            correct_checkMIDS += int(check_MIDS(adjacency_matrix.numpy(),
                                              pred.cpu().numpy(),
                                              sum(torch.split(data.get_example(idx).y, data.num_nodes)[0].cpu().numpy())))

    #print(f'{correct_checkMIDS/len(loader.dataset)*100}%')
    return correct_checkMIDS/len(loader.dataset)*100


times = []
losses = []
trains = []
tests = []

for epoch in range(1, 201):
    start = time.time()

    loss = train()
    losses.append(loss)
    if epoch == 1 or epoch % 1 == 0:
      #train_f1 = test(train_loader)
      #trains.append(train_f1)

      test_f1 = test(test_loader)
      tests.append(test_f1)

      print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
            f'Test: {test_f1:.4f}%, Time: {(time.time() - start):.1f} sec')
    else:
      print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
            f'Time: {(time.time() - start):.1f} sec')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

Plot loss and train and test accuracy

In [ ]:
import matplotlib.pyplot as pp
pp.plot(losses)
pp.show()

In [ ]:
pp.plot(trains)
pp.show()

In [ ]:
pp.plot(tests)
pp.show()

Save/Load a model

In [ ]:
torch.save(model, '/home/jovyan/models/model.pth')

In [ ]:
loaded = torch.load('/home/jovyan/models/3-30_with_real_probabilities_23epochs.pth')

Testing a loaded model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

@torch.no_grad()
def test(loader):
    loaded.eval()
    flag = 0
    correct = 0
    correct_checkMIDS = 0
    for i, data in enumerate(loader):
        for idx in range(data.num_graphs):
          out = loaded(data.get_example(idx).x.type(torch.FloatTensor).to(device), data.get_example(idx).edge_index.to(device))
          #print(out)
          pred = torch.where(out > 0, 1.0, 0.0)
          #print(data.edge_index)

          #correct += int(torch.equal(pred.to(device),data.get_example(idx).y.type(torch.LongTensor).to(device)))  # Derive ratio of correct predictions.

          num_nodes = data.get_example(idx).num_nodes
          adjacency = SparseTensor(row=data.get_example(idx).edge_index[0], col=data.get_example(idx).edge_index[1], sparse_sizes=(num_nodes, num_nodes))
          adjacency_matrix = adjacency.to_dense()
          correct_checkMIDS += int(check_MIDS(adjacency_matrix.numpy(),
                                              pred.cpu().numpy(),
                                              sum(torch.split(data.get_example(idx).y, data.num_nodes)[0].cpu().numpy())))

    #print(f'{correct_checkMIDS/len(loader.dataset)*100}%')
    return correct_checkMIDS/len(loader.dataset)*100

loader = DataLoader(dataset, batch_size=1, shuffle=False)
test(loader)

Visualize wrong predictions

In [ ]:
def check_MIDS(A, candidate, target_value):
    """
    Args:
        - A: adjacency matrix
        - candidate: node labels that are candidate for MIDS (data.y)
        - target_value: known size of the MIDS
    """
    # TODO: This function needs to be adjusted.
    #   - Instead of adjacency matrix, we may pass the edgelist and convert it here

    n = len(candidate)

    # Candidate set is not minimal
    if sum(candidate) > target_value:
        return False

    # Candidate set is not dominating and independent
    if not all((A + np.eye(n)) @ candidate >= 1):
        return False

    for i in range(n):
        for j in range(i+1, n):
            if candidate[i] and candidate[j] and A[i,j]:
                return False



    if sum(candidate) < target_value:
        print(f"Somehow we found an even smaller MIDS: {sum(candidate)}, {target_value}")
        pass

    return True

def examine_pred(data, pred):
    G = pygUtils.to_networkx(data, to_undirected=True)
    nx.draw(G, with_labels=True, node_color=pred, cmap=matplotlib.colormaps["bwr"])
    plt.show()
    print(f'prediction: {pred}')
    print(f'data.y:     {data.y.type(torch.LongTensor)}')

for i in range(500):
    data = dataset[i]
    predik = loaded(data.x.to(device), data.edge_index.to(device))#.argmax(dim=1)
    pred = torch.where(predik > 0, 1.0, 0.0).to(device='cpu')
    num_nodes = data.num_nodes
    adjacency = SparseTensor(row=data.edge_index[0], col=data.edge_index[1], sparse_sizes=(num_nodes, num_nodes))
    adjacency_matrix = adjacency.to_dense()
    is_mids = check_MIDS(adjacency_matrix.numpy(),
                                              pred.cpu().numpy(),
                                              sum(torch.split(data.y, data.num_nodes)[0].cpu().numpy()))
    if not is_mids:
        examine_pred(data.to(device), pred)
        print(f'Prediction: {loaded(data.x.to(device), data.edge_index.to(device)).detach()}')
        #print(data.feature)